In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
pd.set_option('display.max_columns', None)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

In [2]:
ddos_data = pd.read_csv('DDoS_fnl.csv', nrows=100000)
ddos_data

C:\Users\akqqk\AppData\Local\Temp\ipykernel_18388\3785896865.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79) have mixed types. Specify dtype option on import or set low_memory=False.
  ddos_data = pd.read_csv('DDoS_fnl.csv', nrows=100000)


,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,853,12183,17,1,2,0,2832.0,0.0,1416.0,1416.0,1416.0,0.0,0.0,0.0,0.0,0.0,2832000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2000000.0,0.0,1416.0,1416.0,1416.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2124.0,1416.0,0.0,0,0,0,0,0,0,0,2,2832,0,0,-1,-1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
1,1000,27431,17,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,2944000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2000000.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,0,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
2,681,32152,17,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,1472000000.0,1000000.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,2076,0,1000000.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,2076,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,1038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
3,855,59107,17,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,2944000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2000000.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,0,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
4,675,50606,17,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,2944000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2000000.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,0,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,810,33895,17,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,2944000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2125437944,0,2000000.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2125437944,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1062718972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
99996,955,16503,17,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,2944000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,-2125437944,0,2000000.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,-2125437944,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,-1062718972,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
99997,900,59535,17,1,2,0,2944.0,0.0,1472.0,

In [3]:
normal_data = pd.read_csv('Normal_fnl.csv', nrows=100000)
normal_data

C:\Users\akqqk\AppData\Local\Temp\ipykernel_18388\2591145696.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79) have mixed types. Specify dtype option on import or set low_memory=False.
  normal_data = pd.read_csv('Normal_fnl.csv', nrows=100000)


,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54993.0,80.0,6.0,115846936.0,16.0,14.0,664.0,764.0,580.0,0.0,41.5,143.607799,758.0,0.0,54.571429,202.466983,12.32661,0.258962,3994721.931,4859656.363,10000000.0,79.0,116000000.0,7723129.067,4115825.281,10000000.0,92.0,110000000.0,8494802.769,3751513.839,10000000.0,37285.0,0.0,0.0,0.0,0.0,332.0,424.0,0.138113,0.120849,0.0,758.0,46.064516,167.901149,28190.7957,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,47.6,41.5,54.571429,332.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,664.0,14.0,764.0,8192.0,344.0,15.0,20.0,61975.45455,82825.78542,311704.0,36878.0,9977345.818,60059.81721,10000000.0,9796817.0,BENIGN
1,55032.0,53.0,17.0,192.0,2.0,2.0,100.0,132.0,50.0,50.0,50.0,0.0,66.0,66.0,66.0,0.0,1208333.333,20833.33333,64.0,27.712813,96.0,48.0,48.0,48.0,0.0,48.0,48.0,48.0,48.0,0.0,48.0,48.0,0.0,0.0,0.0,0.0,40.0,40.0,10416.66667,10416.66667,50.0,66.0,56.4,8.763561,76.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,70.5,50.0,66.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,100.0,2.0,132.0,-1.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,53834.0,465.0,6.0,128.0,3.0,0.0,49.0,0.0,37.0,6.0,16.333333,17.897858,0.0,0.0,0.0,0.0,382812.5,23437.5,64.0,24.041631,81.0,47.0,128.0,64.0,24.041631,81.0,47.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,0.0,23437.5,0.0,6.0,37.0,13.75,15.5,240.25,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,18.333333,16.333333,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,49.0,0.0,0.0,257.0,-1.0,2.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,62922.0,53.0,17.0,198.0,2.0,2.0,168.0,264.0,84.0,84.0,84.0,0.0,132.0,132.0,132.0,0.0,2181818.182,20202.0202,66.0,72.02083,145.0,4.0,4.0,4.0,0.0,4.0,4.0,49.0,49.0,0.0,49.0,49.0,0.0,0.0,0.0,0.0,40.0,40.0,10101.0101,10101.0101,84.0,132.0,103.2,26.290683,691.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,129.0,84.0,132.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,168.0,2.0,264.0,-1.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,40110.0,80.0,6.0,15725.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.186009,15725.0,0.0,15725.0,15725.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,63.593005,63.593005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,320.0,31.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,443.0,3307.0,6.0,3.0,2.0,0.0,37.0,0.0,31.0,6.0,18.5,17.67767,0.0,0.0,0.0,0.0,12300000.0,666666.6667,3.0,0.0,3.0,3.0,3.0,3.0,

In [4]:
data = pd.concat([ddos_data, normal_data], axis=0)
data

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,853,12183,17,1,2,0,2832.0,0.0,1416.0,1416.0,1416.0,0.0,0.0,0.0,0.0,0.0,2832000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2000000.0,0.0,1416.0,1416.0,1416.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2124.0,1416.0,0.0,0,0,0,0,0,0,0,2,2832,0,0,-1,-1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
1,1000,27431,17,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,2944000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2000000.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,0,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
2,681,32152,17,2,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,1472000000.0,1000000.0,2.0,0.0,2.0,2.0,2.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,2076,0,1000000.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,2076,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,1038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
3,855,59107,17,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,2944000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2000000.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,0,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
4,675,50606,17,1,2,0,2944.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0.0,0.0,0.0,2944000000.0,2000000.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,2000000.0,0.0,1472.0,1472.0,1472.0,0.0,0.0,0,0,0,0,0,0,0,0,0.0,2208.0,1472.0,0.0,0,0,0,0,0,0,0,2,2944,0,0,-1,-1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,DDoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,443.0,3307.0,6.0,3.0,2.0,0.0,37.0,0.0,31.0,6.0,18.5,17.67767,0.0,0.0,0.0,0.0,12300000.0,666666.6667,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,40.0,0.0,666666.6667,0.0,6.0,31.0,22.666667,14.433757,208.333333,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,34.0,18.5,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,37.0,0.0,0.0,946.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
99996,46799.0,443.0,6.0,5803751.0,9.0,6.0,375.0,4470.0,182.0,0.0,41.666667,74.752926,1418.0,0.0,745.0,716.239904,834.804939,2.584535,414553.6429,1464095.506,5499446.0,30.0,5803751.0,725468.875,1929650.911,5499446.0,48.0,229073.0,45814.6,81491.60065,188306.0,48.0,0.0,0.0,0.0,0.0,296.0,200.0,1.550721,1.033814,0.0,1418.0,302.8125,547.013128,299223.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200000 entries, 0 to 99999
Data columns (total 81 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   Source Port                  200000 non-null  object
 1   Destination Port             200000 non-null  object
 2   Protocol                     200000 non-null  object
 3   Flow Duration                200000 non-null  object
 4   Total Fwd Packets            200000 non-null  object
 5   Total Backward Packets       200000 non-null  object
 6   Total Length of Fwd Packets  200000 non-null  object
 7   Total Length of Bwd Packets  200000 non-null  object
 8   Fwd Packet Length Max        200000 non-null  object
 9   Fwd Packet Length Min        200000 non-null  object
 10  Fwd Packet Length Mean       200000 non-null  object
 11  Fwd Packet Length Std        200000 non-null  object
 12  Bwd Packet Length Max        200000 non-null  object
 13  Bwd Packet Length Mi

In [6]:
data['Label']

0          DDoS
1          DDoS
2          DDoS
3          DDoS
4          DDoS
          ...  
99995    BENIGN
99996    BENIGN
99997    BENIGN
99998    BENIGN
99999    BENIGN
Name: Label, Length: 200000, dtype: object

In [7]:
data.isnull().sum()

Source Port          0
Destination Port     0
Protocol             0
Flow Duration        0
Total Fwd Packets    0
                    ..
Idle Mean            0
Idle Std             0
Idle Max             0
Idle Min             0
Label                0
Length: 81, dtype: int64

In [8]:
data['Label'].value_counts()

Label
DDoS      100000
BENIGN     99994
Label          6
Name: count, dtype: int64

In [9]:
old_value = 'Label'
new_value = 'BENIGN'

# Replace the value in the specified column
data['Label'] = data['Label'].replace(old_value, new_value)

In [10]:
data['Label'].value_counts()

Label
DDoS      100000
BENIGN    100000
Name: count, dtype: int64

In [11]:
data = data.drop(['Fwd Header Length.1'], axis=1)

In [12]:
X = data.drop(['Label'], axis=1)
y = data['Label']

In [13]:
X = X.apply(pd.to_numeric, errors='coerce')

In [14]:
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

In [15]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 79 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Source Port                  200000 non-null  float64
 1   Destination Port             200000 non-null  float64
 2   Protocol                     200000 non-null  float64
 3   Flow Duration                200000 non-null  float64
 4   Total Fwd Packets            200000 non-null  float64
 5   Total Backward Packets       200000 non-null  float64
 6   Total Length of Fwd Packets  200000 non-null  float64
 7   Total Length of Bwd Packets  200000 non-null  float64
 8   Fwd Packet Length Max        200000 non-null  float64
 9   Fwd Packet Length Min        200000 non-null  float64
 10  Fwd Packet Length Mean       200000 non-null  float64
 11  Fwd Packet Length Std        200000 non-null  float64
 12  Bwd Packet Length Max        200000 non-null  float64
 13 

In [16]:
X.describe()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,200000.000000,200000.000000,200000.000000,2.000000e+05,200000.000000,200000.000000,2.000000e+05,2.000000e+05,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,200000.000000,200000.0,200000.000000,200000.0,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,200000.000000,200000.000000,200000.000000,200000.000000,2.000000e+05,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.0,200000.0,200000.0,200000.0,200000.0,200000.0,200000.000000,2.000000e+05,200000.000000,2.000000e+05,200000.000000,200000.00000,200000.000000,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05,2.000000e+05
mean,20449.218609,21038.569968,13.903912,5.665838e+06,8.089778,5.975493,1.784180e+03,9.523302e+03,845.547517,742.976953,764.464083,36.792676,199.387318,25.243106,80.694702,61.761699,1.316725e+09,9.321430e+05,4.396828e+05,7.817862e+05,2.241362e+06,6.500369e+04,5.503578e+06,9.201186e+05,6.334603e+05,2.164863e+06,5.372210e+05,5.118463e+06,8.601101e+05,5.061014e+05,1.878445e+06,5.262449e+05,0.028457,0.0,0.000045,0.0,-5.700214e+07,-2.153630e+04,9.289652e+05,3.207833e+03,741.641810,978.089478,786.193465,73.661146,5.670183e+04,0.008641,0.026687,0.002025,0.127136,0.140727,0.061780,0.002765,0.000185,0.354035,1159.131257,764.464083,80.694702,0.0,0.0,0.0,0.0,0.0,0.0,8.089778,1.784180e+03,5.975493,9.523101e+03,3479.197027,1209.53074,4.138982,-2.858899e+07,3.828122e+04,2.326667e+04,8.065553e+04,2.544847e+04,1.877044e+06,1.389961e+05,1.979706e+06,1.755332e+06
std,25953.213159,22561.264510,4.952565,2.215811e+07,758.926871,825.562260,4.849884e+03,2.009498e+06,829.061457,722.689225,714.073775,218.452070,609.126661,57.541917,216.733850,204.242201,1.400854e+09,9.425357e+05,2.811696e+06,4.140579e+06,1.042849e+07,1.753753e+06,2.202713e+07,6.413027e+06,3.384730e+06,1.046850e+07,6.137073e+06,2.133303e+07,6.344665e+06,3.024423e+06,9.860381e+06,6.084023e+06,0.166269,0.0,0.006708,0.0,3.433903e+08,6.723844e+06,9.444977e+05,2.836267e+04,722.847588,897.992963,690.664881,226.433096,3.998561e+05,0.092549,0.161161,0.044955,0.333112,0.347726,0.240746,0.052511,0.013600,0.595035,1046.036743,714.073775,216.733850,0.0,0.0,0.0,0.0,0.0,0.0,758.926871,4.849884e+03,825.562260,2.009493e+06,11191.909421,6714.26464,606.333200,1.719392e+08,4.291301e+05,3.163056e+05,7.619972e

In [17]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Standardize the data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
X_train

array([[0.00122072, 0.97155718, 0.35294118, ..., 0.        , 0.        ,
        0.        ],
       [0.01428244, 0.14290074, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.647776  , 0.01625086, 0.35294118, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00122072, 0.80976577, 0.35294118, ..., 0.        , 0.        ,
        0.        ],
       [0.00122072, 0.82467384, 0.35294118, ..., 0.        , 0.09833333,
        0.09833333],
       [0.86137179, 0.00675975, 0.35294118, ..., 0.        , 0.        ,
        0.        ]])

In [21]:
X_train.shape[1]

79

In [22]:
# Reshape the data for LSTM input (assuming your data is sequential)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [23]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(1, X_train.shape[2])))
model.add(Dropout(0.2))  # Adding dropout for regularization  # Adding dropout for regularization
model.add(LSTM(32))  # Adding a third LSTM layer
model.add(Dropout(0.2))  # Adding dropout for regularization
model.add(Dense(1, activation='sigmoid'))

In [24]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/3


1250/1250 [==============================] - 16s 8ms/step - loss: 0.0322 - accuracy: 0.9979 - val_loss: 8.1346e-04 - val_accuracy: 0.9999
Epoch 2/3
1250/1250 [==============================] - 8s 6ms/step - loss: 0.0015 - accuracy: 0.9998 - val_loss: 7.4613e-04 - val_accuracy: 0.9999
Epoch 3/3
1250/1250 [==============================] - 8s 6ms/step - loss: 0.0014 - accuracy: 0.9998 - val_loss: 8.8445e-04 - val_accuracy: 0.9998


In [26]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

1250/1250 [==============================] - 4s 3ms/step - loss: 8.8445e-04 - accuracy: 0.9998
Test Accuracy: 0.999750018119812


In [27]:
# Save the model to a file
model.save('lstm_model.h5')

C:\Users\akqqk\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
